## Feature Scaling: Estandarización Z-Score y escalado Min-Max 

- Acerca de la estandarización
- Sobre el escalado Min-Max y la estandarización
- ¿Estandarización o escalado Min-max?
- Estándarización o escalado Min-max, cómo podemos hacerlo en scikit-learn
- Acercamientos de abajo a arriba, Down-top
- El efecto de la estandarización en un PCA con un patrón de clasificación

<br>
<br>

### Acerca de la estandarización

El resultado de la **estandarización** (o **normalización de la puntuación Z**) es que las características se reescalarán para que tengan las propiedades de una distribución normal estándar con:

$\mu = 0$ and $\sigma = 1$

donde $\mu$ es la media (promedio) y $\sigma$ es la desviación estándar de la media; las puntuaciones estándar (también llamadas puntuaciones ***z***) de las muestras se calculan de la siguiente manera:

$$z = \frac{x - \mu}{\sigma}$$

La normalización de las características/features para que se centren en torno a 0 con una desviación estándar de 1 no sólo es importante si estamos comparando las mediciones que tienen diferentes unidades, pero también es un requisito general para muchos algoritmos de aprendizaje automático. Intuitivamente, podemos pensar en el descenso de gradiente como un ejemplo destacado (un algoritmo de optimización utilizado a menudo en regresión logística, SVMs, perceptrones, redes neuronales, etc.); con características en diferentes escalas, ciertos pesos pueden actualizarse más rápido que otros ya que los valores de las características $x_j$ juegan un papel en las actualizaciones de los pesos:

$$\Delta w_j = - \eta \frac{\partial J}{\partial w_j} = \eta \sum_i (t^{(i)} - o^{(i)})x^{(i)}_{j},$$

De modo que:

$$w_j := w_j + \Delta w_j,$$
donde $\eta$ es la tasa de aprendizaje, $t$ la etiqueta de la clase objetivo y $o$ la salida real.
Otros ejemplos intuitivos son los algoritmos K-Nearest Neighbor y los algoritmos de agrupación que utilizan, por ejemplo, medidas de distancia euclidiana; de hecho, los clasificadores basados en árboles son probablemente los únicos clasificadores en los que el escalado de características no supone ninguna diferencia.



Citando el [`scikit-learn`](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) documentation:

*"Standardization of a dataset is a common requirement for many machine learning estimators: they might behave badly if the individual feature do not more or less look like standard normally distributed data (e.g. Gaussian with 0 mean and unit variance)."*

<br>
<br>

<a id='About-Min-Max-scaling-normalization'></a>

### Sobre el escalado Min-Max y la estandarización

Un enfoque alternativo a la normalización (o estandarización) de la puntuación Z es el llamado **escalado Mín-Máx** (a menudo también llamado simplemente "normalización", una causa común de ambigüedades).  
En este enfoque, los datos se escalan a un intervalo fijo, normalmente de 0 a 1. El coste de tener este intervalo limitado es que los datos se escalan a un intervalo fijo.  
El coste de tener este rango acotado -en contraste con la normalización- es que acabaremos con desviaciones estándar más pequeñas, lo que puede suprimir el efecto de los valores atípicos.

Un escalado Mín-Máx se realiza normalmente mediante la siguiente ecuación:

\begin{equation} X_{norm} = \frac{X - X_{min}}{X_{max}-X_{min}} \end{equation}

<br>
<br>

### ¿Estandarización o escalado Min-max?

"¿Estandarización o escalado Min-Max?" - No hay una respuesta obvia a esta pregunta: realmente depende de la aplicación. 

Por ejemplo, en los análisis de agrupamiento, la normalización puede ser especialmente crucial para comparar similitudes entre características basadas en determinadas medidas de distancia. Otro ejemplo destacado es el análisis de componentes principales, en el que solemos preferir la normalización al escalado Min-Max, ya que nos interesan los componentes que maximizan la varianza (dependiendo de la pregunta y de si el PCA calcula los componentes a través de la matriz de correlaciones en lugar de la matriz de covarianzas; [pero más sobre PCA en mi artículo anterior](http://sebastianraschka.com/Articles/2014_pca_step_by_step.html)).

Sin embargo, ¡esto no significa que el escalado Min-Max no sea útil en absoluto! Una aplicación popular es el procesamiento de imágenes, donde las intensidades de los píxeles tienen que normalizarse para ajustarse a un cierto rango (es decir, de 0 a 255 para la gama de colores RGB). Además, los algoritmos típicos de redes neuronales requieren datos en una escala de 0 a 1.

<br>
<br>

## Entandarización y escalado, cómo se hace en Numpy

Por supuesto, podríamos hacer uso de las capacidades de vectorización de NumPy para calcular las puntuaciones z para la normalización y para normalizar los datos utilizando las ecuaciones que se mencionaron en las secciones anteriores. Sin embargo, hay un enfoque aún más conveniente utilizando el módulo de preprocesamiento de una de las bibliotecas de aprendizaje automático de código abierto de Python [scikit-learn](http://scikit-learn.org ).

<br>
<br>

Para los siguientes ejemplos y debates, utilizaremos el conjunto de datos gratuito "Wine" depositado en el repositorio de aprendizaje automático de la UCI  
(http://archive.ics.uci.edu/ml/datasets/Wine)

<br>

<font size="1">
**Referencia:**  
Forina, M. et al, PARVUS - An Extendible Package for Data
Exploration, Classification and Correlation. Institute of Pharmaceutical
and Food Analysis and Technologies, Via Brigata Salerno, 
16147 Genoa, Italy.

Bache, K. & Lichman, M. (2013). UCI Machine Learning Repository [http://archive.ics.uci.edu/ml]. Irvine, CA: University of California, School of Information and Computer Science.

</font>

El conjunto de datos Wine consta de 3 clases diferentes, en las que cada fila corresponde a una muestra de vino concreta.

Las etiquetas de clase (1, 2, 3) figuran en la primera columna, y las columnas 2-14 corresponden a 13 atributos (características) diferentes:

1) Alcohol  
2) Ácido málico  
...

#### Cargando el dataset

In [ ]:
import pandas as pd
import numpy as np

df = pd.io.parsers.read_csv(
    'https://raw.githubusercontent.com/rasbt/pattern_classification/master/data/wine_data.csv', 
     header=None,
     usecols=[0,1,2]
    )

df.columns=['Class label', 'Alcohol', 'Malic acid']

df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

Como podemos ver en la tabla anterior, las características **Alcohol** (porcentaje/volumen) y **Ácido málico** (g/l) se miden en escalas diferentes, por lo que ***Escalado de características*** es necesario importante antes de cualquier comparación o combinación de estos datos.  



#### Estandardarización y Min-Max scaling

In [ ]:
df[['Alcohol', 'Malic acid']].describe().loc[['mean', 'std']]

In [ ]:
df[['Alcohol', 'Malic acid']].describe().loc[['min', 'max']]

In [ ]:
from sklearn import preprocessing

std_scale = preprocessing.StandardScaler().fit(df[['Alcohol', 'Malic acid']])
df_std = std_scale.transform(df[['Alcohol', 'Malic acid']])

minmax_scale = preprocessing.MinMaxScaler().fit(df[['Alcohol', 'Malic acid']])
df_minmax = minmax_scale.transform(df[['Alcohol', 'Malic acid']])

In [ ]:
df_std

In [ ]:
print('Mean after standardization:\nAlcohol={:.2f}, Malic acid={:.2f}'
      .format(df_std[:,0].mean(), df_std[:,1].mean()))

      
print('\nStandard deviation after standardization:\nAlcohol={:.2f}, Malic acid={:.2f}'
      .format(df_std[:,0].std(), df_std[:,1].std()))

In [ ]:
print('Min-value after min-max scaling:\nAlcohol={:.2f}, Malic acid={:.2f}'
      .format(df_minmax[:,0].min(), df_minmax[:,1].min()))
print('\nMax-value after min-max scaling:\nAlcohol={:.2f}, Malic acid={:.2f}'
      .format(df_minmax[:,0].max(), df_minmax[:,1].max()))

#### Vamos a pintar!

In [ ]:
%matplotlib inline

In [ ]:
from matplotlib import pyplot as plt

def plot():
    plt.figure(figsize=(8,6))

    plt.scatter(df['Alcohol'], df['Malic acid'], 
            color='green', label='input scale', alpha=0.5)

    plt.scatter(df_std[:,0], df_std[:,1], color='red', 
            label='Standardized [$N  (\mu=0, \; \sigma=1)$]', alpha=0.3)

    plt.scatter(df_minmax[:,0], df_minmax[:,1], 
            color='blue', label='min-max scaled [min=0, max=1]', alpha=0.3)

    plt.title('Alcohol and Malic Acid content of the wine dataset')
    plt.xlabel('Alcohol')
    plt.ylabel('Malic Acid')
    plt.legend(loc='upper left')
    plt.grid()
    
    plt.tight_layout()

plot()
plt.show()

<br>
<br>

El gráfico anterior incluye los puntos de datos del vino en las tres escalas diferentes: la escala de entrada en la que se midió el contenido de alcohol en volumen-porcentaje (verde), las características normalizadas (rojo) y las características normalizadas (azul).
En el siguiente gráfico, ampliaremos los tres ejes-escalas.

<br>
<br>

In [ ]:
fig, ax = plt.subplots(3, figsize=(6,14))

for a,d,l in zip(range(len(ax)), 
               (df[['Alcohol', 'Malic acid']].values, df_std, df_minmax),
               ('Input scale', 
                'Standardized [$N  (\mu=0, \; \sigma=1)$]', 
                'min-max scaled [min=0, max=1]')
                ):
    for i,c in zip(range(1,4), ('red', 'blue', 'green')):
        ax[a].scatter(d[df['Class label'].values == i, 0], 
                  d[df['Class label'].values == i, 1],
                  alpha=0.5,
                  color=c,
                  label='Class %s' %i
                  )
    ax[a].set_title(l)
    ax[a].set_xlabel('Alcohol')
    ax[a].set_ylabel('Malic Acid')
    ax[a].legend(loc='upper left')
    ax[a].grid()
    
plt.tight_layout()

plt.show()

### Comparar características con diferentes escalas

In [ ]:
from scipy import stats
price_madrid = stats.gamma.rvs(1, size=5000)*100000

plt.hist(price_madrid, 70, histtype="stepfilled", alpha=.7);

In [ ]:
# Precios de casas en diferentes monedas
from scipy import stats
import matplotlib.pyplot as plt
price_madrid = stats.gamma.rvs(1, size=5000)*100000
price_london = stats.gamma(5).rvs(5000)*100000*0.87
price_spetesbourg = stats.gamma(5).rvs(5000)*100000*90.23

plt.hist(price_madrid, 70, alpha = .7)
plt.hist(price_london, 70, alpha = .7)
plt.hist(price_spetesbourg, 70, alpha = .7);

In [ ]:
df = pd.DataFrame({'Madrid': price_madrid,
                  'London': price_london,
                  'Saint Petersburg': price_spetesbourg})

minmax_scale = preprocessing.MinMaxScaler().fit(df)
df_minmax = minmax_scale.transform(df)

plt.hist(df_minmax[:, 0], 70, histtype="stepfilled", alpha=.7, label='Madrid')
plt.hist(df_minmax[:, 1], 70, histtype="stepfilled", alpha=.7, label='London')
plt.hist(df_minmax[:, 2], 70, histtype="stepfilled", alpha=.7, label='Saint Petersburg')
plt.legend();

## Acercamientos de arriba a abajo

Por supuesto, también podemos codificar las ecuaciones para la estandarización y el escalado 0-1 Min-Max "manualmente". Sin embargo, los métodos de scikit-learn siguen siendo útiles si se trabaja con conjuntos de datos de prueba y de entrenamiento y se desea escalarlos por igual.

E.g., 
<pre>
std_scale = preprocessing.StandardScaler().fit(X_train)
X_train = std_scale.transform(X_train)
X_test = std_scale.transform(X_test)
</pre>

A continuación, realizaremos los cálculos utilizando código Python "puro", y una solución NumPy más conveniente, que es especialmente útil si intentamos transformar una matriz entera.

<br>
<br>

Sólo para recordar las ecuaciones que estamos utilizando:

Normalización:

\begin{equation} z = \frac{x - \mu}{\sigma}\end{equation} 


Con media:

\begin{equation}\mu = \frac{1}{N} \sum_{i=1}^N (x_i)\end{equation}

Con desviación estándar:

\begin{equation}\sigma = \sqrt{\frac{1}{N} \sum_{i=1}^N (x_i - \mu)^2}\end{equation}


Escalado Min-Max:

\begin{equation} X_{norm} = \frac{X - X_{min}}{X_{max}-X_{min}} \end{equation}

### Vamos con python:

In [ ]:
# Standardization

x = [1,4,5,6,6,2,3]
mean = sum(x)/len(x)
std_dev = (1/len(x) * sum([ (x_i - mean)**2 for x_i in x]))**0.5

z_scores = [(x_i - mean)/std_dev for x_i in x]

# Min-Max scaling

minmax = [(x_i - min(x)) / (max(x) - min(x)) for x_i in x]

<br>
<br>

### NumPy

In [ ]:
import numpy as np

# Standardization

x_np = np.asarray(x)
z_scores_np = (x_np - x_np.mean()) / x_np.std()

# Min-Max scaling

np_minmax = (x_np - x_np.min()) / (x_np.max() - x_np.min())

<br>
<br>

### Visualización:

Sólo para asegurarnos de que nuestro código funciona correctamente, vamos a trazar los resultados a través de matplotlib.

In [ ]:
%matplotlib inline

In [ ]:
from matplotlib import pyplot as plt

fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(nrows=2, ncols=2, figsize=(10,5))

y_pos = [0 for i in range(len(x))]

ax1.scatter(z_scores, y_pos, color='g')
ax1.set_title('Python standardization', color='g')

ax2.scatter(minmax, y_pos, color='g')
ax2.set_title('Python Min-Max scaling', color='g')

ax3.scatter(z_scores_np, y_pos, color='b')
ax3.set_title('Python NumPy standardization', color='b')

ax4.scatter(np_minmax, y_pos, color='b')
ax4.set_title('Python NumPy Min-Max scaling', color='b')
    
plt.tight_layout()

for ax in (ax1, ax2, ax3, ax4):
    ax.get_yaxis().set_visible(False)
    ax.grid()

plt.show()

<br>
<br>

# Binning

In [ ]:
import pandas as pd

fcc_survey_df = pd.read_csv('fcc_2016_coder_survey_subset.csv', encoding='utf-8')
fcc_survey_df[['ID.x', 'EmploymentField', 'Age', 'Income']].head()

## Binning con cortes fijos

### Distribución de la edad de los desarrolladores:

In [ ]:
fig, ax = plt.subplots()
fcc_survey_df['Age'].hist(color='#A9C5D3')
ax.set_title('Developer Age Histogram', fontsize=12)
ax.set_xlabel('Age', fontsize=12)
ax.set_ylabel('Frequency', fontsize=12);

### Binning basado en redondeo:

``` 
Age Range: Bin
---------------
 0 -  9  : 0
10 - 19  : 1
20 - 29  : 2
30 - 39  : 3
40 - 49  : 4
50 - 59  : 5
60 - 69  : 6
  ... and so on
```

In [ ]:
fcc_survey_df['Age_bin_round'] = np.array(np.floor(np.array(fcc_survey_df['Age']) / 10.))
fcc_survey_df[['ID.x', 'Age', 'Age_bin_round']].iloc[1071:1076]

### Binning basado en rangos personalizados:

``` 
Age Range : Bin
---------------
 0 -  15  : 1
16 -  30  : 2
31 -  45  : 3
46 -  60  : 4
61 -  75  : 5
75 - 100  : 6
```

In [ ]:
fcc_survey_df['Age'].unique()

In [ ]:
bin_ranges = [0, 15, 30, 45, 60, 75, 100]
bin_names = [1, 2, 3, 4, 5, 6]

fcc_survey_df['Age_bin_custom_range'] = pd.cut(np.array(fcc_survey_df['Age']), 
                                               bins=bin_ranges)
fcc_survey_df['Age_bin_custom_label'] = pd.cut(np.array(fcc_survey_df['Age']), 
                                               bins=bin_ranges, labels=bin_names)
fcc_survey_df[['ID.x', 'Age', 'Age_bin_round', 
               'Age_bin_custom_range', 'Age_bin_custom_label']].iloc[1071:1076]

In [ ]:
def bin_ages(x):
    if x >= 0 and x <=15:
        return 1
    pass

fcc_survey_df['Age_bin_custom_label'] = fcc_survey_df['Age'].apply(bin_ages)